<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=4ac9f40186701e3094ae35aaaa5c19fceccc6ba62045b957e845cc926344a1ab
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-12 13:19:52
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.40 C
-------------------
Today PnL: 1.07 K (0.01%)
Current PnL: -28.63 L (-18.5%)
CY Booked + Current PnL: -14.34 L (-9.27%)
-------------------
Total profit:  1.32 L
Total loss:  -29.96 L
-------------------
Total Booked + Current PnL: 12.64 L (9.88%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.23%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 93.27 L (66.79%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 4.95%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,0.06,-16.75,20.24,0.09,16980.0,-16885.0,83892.0,94.66,42.0,M-SC,2.37,253.0,-0.99,0.62,10.22,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-0.04,-12.51,14.32,0.02,21869.0,-21841.0,152718.0,132.70,52.0,M-SC,15.82,234.0,-1.00,1.13,32.84,OX40N,NTT,PAINTS
50,MASFIN,398.61,0.73,-1.32,23.68,22.05,22896.0,-1290.0,96690.0,-15.09,58.0,H-SC,6.96,164.0,-0.06,0.72,40.55,XR,ATH,FINANCE
82,VOLTAS,1530.00,0.71,6.63,12.25,19.69,25045.0,12708.0,204450.0,-3.84,51.0,X-MC,2.58,78.0,0.51,1.52,13.88,XY25,NTT,AC
43,ITC,452.00,-0.56,-2.11,12.82,10.44,25117.0,-4220.0,195918.0,-41.56,42.0,X-LC,1.84,5.0,-0.17,1.45,3.34,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,KPIGREEN,730.66,4.27,-16.09,75.13,46.95,78987.0,-20163.0,105134.0,-22.59,40.0,H-SC,6.81,154.0,-0.26,0.78,31.71,MH,ATH,POWER
10,BANDHANBNK,400.00,2.90,-28.08,167.56,92.42,335673.0,-78229.0,200330.0,40.80,50.0,H-SC,8.73,170.0,-0.23,1.49,15.77,XY24,NTT,BANKS
22,DIXON,18931.72,2.18,-14.77,42.65,21.58,50941.0,-20700.0,119439.0,-57.05,35.0,X-MC,7.74,56.0,-0.41,0.89,7.45,X40N,ATH,IT
46,JSWINFRA,342.00,1.83,-8.39,25.78,15.23,47315.0,-16815.0,183532.0,-21.67,45.0,X-MC,8.88,66.0,-0.36,1.36,21.15,X40N,NTT,REALTY
69,SURYODAY,216.00,1.49,-21.42,56.57,23.03,79605.0,-38352.0,140719.0,53.51,47.0,H-SC,8.98,167.0,-0.48,1.04,39.62,XR,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-2.67,-16.41,50.15,25.51,78013.0,-30535.0,155560.0,-4.56,41.0,H-LC,4.18,49.0,-0.39,1.15,14.64,AR,ATH,ELECTRICAL
57,RAJOOENG,143.10,-2.61,-38.36,126.50,39.61,79923.0,-39320.0,63180.0,-58.33,23.0,H-SC,28.10,135.0,-0.49,0.47,0.00,AR,ATH,MISC
18,COFFEEDAY,80.00,-2.42,-44.86,145.32,35.27,90986.0,-50938.0,62611.0,-57.75,32.0,L-SC,11.07,270.0,-0.56,0.46,53.24,XR,NTT,HOTELS
26,GILLETTE,11206.78,-2.31,-4.41,38.53,32.42,93512.0,-11196.0,242700.0,-21.44,38.0,X-SC,9.18,85.0,-0.12,1.80,10.02,X40,ATH,FMCG
33,HINDUNILVR,2922.00,-1.90,-12.01,29.18,13.67,66002.0,-30873.0,226190.0,-18.61,25.0,X-LC,2.42,14.0,-0.47,1.68,6.78,XY25,NTT,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.70,-7.11,123.83,107.91,169828.0,-10498.0,137146.0,-24.60,38.0,M-SC,12.53,216.0,-0.06,1.02,2.26,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.04,-12.51,14.32,0.02,21869.0,-21841.0,152718.0,132.70,52.0,M-SC,15.82,234.0,-1.00,1.13,32.84,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.06,-16.75,20.24,0.09,16980.0,-16885.0,83892.0,94.66,42.0,M-SC,2.37,253.0,-0.99,0.62,10.22,OX40N,NTT,DURABLES
47,KANSAINER,340.00,0.04,-25.46,52.23,13.47,104985.0,-68661.0,201006.0,-69.28,31.0,H-SC,4.70,158.0,-0.65,1.49,2.21,XY24,NTT,PAINTS
66,SIS,528.00,-0.03,-25.52,62.81,21.26,51941.0,-28336.0,82696.0,1949.22,43.0,H-SC,4.79,166.0,-0.55,0.61,11.73,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.35,6.71,61.44,72.27,111090.0,11363.0,180811.0,-8.35,68.0,M-LC,2.02,99.0,0.1,1.34,13.3,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.35,6.71,61.44,72.27,111090.0,11363.0,180811.0,-8.35,68.0,M-LC,2.02,99.0,0.10,1.34,13.30,XR,NTT,IT
38,INDIAMART,4810.62,-0.77,-4.54,116.55,106.72,137226.0,-5596.0,117740.0,-52.73,30.0,H-SC,2.33,139.0,-0.04,0.87,17.78,AR,ATH,MISC
50,MASFIN,398.61,0.73,-1.32,23.68,22.05,22896.0,-1290.0,96690.0,-15.09,58.0,H-SC,6.96,164.0,-0.06,0.72,40.55,XR,ATH,FINANCE
85,ZYDUSLIFE,1286.17,0.56,-2.17,38.63,35.63,79204.0,-4541.0,205033.0,-15.85,41.0,H-MC,3.11,119.0,-0.06,1.52,14.14,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.70,-7.11,123.83,107.91,169828.0,-10498.0,137146.0,-24.60,38.0,M-SC,12.53,216.0,-0.06,1.02,2.26,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.18,-21.77,89.75,48.45,140787.0,-43640.0,156866.0,-27.39,24.0,X-MC,12.70,64.0,-0.31,1.16,2.14,X40N,ATH,FINANCE
33,HINDUNILVR,2922.00,-1.90,-12.01,29.18,13.67,66002.0,-30873.0,226190.0,-18.61,25.0,X-LC,2.42,14.0,-0.47,1.68,6.78,XY25,NTT,FMCG
75,TMPV,600.00,-0.14,-17.97,73.34,42.18,166282.0,-49682.0,226728.0,-26.74,29.0,X-LC,6.39,3.0,-0.30,1.68,0.80,XY24,NTT,AUTO
23,DMART,5391.45,-0.44,-9.36,41.36,28.12,78876.0,-19702.0,190705.0,-21.84,31.0,X-LC,5.24,19.0,-0.25,1.42,12.05,X40N,ATH,FMCG
53,PAGEIND,51605.08,-0.42,-9.83,39.81,26.07,58775.0,-16100.0,147640.0,-33.64,32.0,X-MC,12.41,55.0,-0.27,1.10,0.59,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.0,-0.07,-80.18,529.73,24.79,264383.0,-201947.0,49909.0,-81.09,45.0,X-SC,0.94,86.0,-0.76,0.37,22.01,X40N,NTT,MISC
43,ITC,452.0,-0.56,-2.11,12.82,10.44,25117.0,-4220.0,195918.0,-41.56,42.0,X-LC,1.84,5.0,-0.17,1.45,3.34,X40,NTT,FMCG
8,AWL,485.0,0.42,-21.37,95.21,53.50,225905.0,-64481.0,237270.0,-61.97,35.0,X-MC,2.09,58.0,-0.29,1.76,5.39,XY24,NTT,FMCG
36,ICICIPRULI,790.0,0.95,6.42,23.07,30.97,43389.0,11339.0,188077.0,-18.62,62.0,X-MC,2.12,75.0,0.26,1.40,19.78,X40,ATH,INSURANCE
3,ACC,3906.0,0.03,-25.22,119.51,64.15,212656.0,-60011.0,177940.0,-56.31,34.0,X-SC,2.27,82.0,-0.28,1.32,0.03,XY24,BTT,CEMENT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,0.03,-25.22,119.51,64.15,212656.0,-60011.0,177940.0,-56.31,34.0,X-SC,2.27,82.0,-0.28,1.32,0.03,XY24,BTT,CEMENT
54,PGHH,17907.65,0.30,-5.89,42.12,33.76,79607.0,-11820.0,189000.0,-33.58,35.0,X-MC,5.35,57.0,-0.15,1.40,0.30,X40,ATH,FMCG
53,PAGEIND,51605.08,-0.42,-9.83,39.81,26.07,58775.0,-16100.0,147640.0,-33.64,32.0,X-MC,12.41,55.0,-0.27,1.10,0.59,X40,ATH,APPARELS
75,TMPV,600.00,-0.14,-17.97,73.34,42.18,166282.0,-49682.0,226728.0,-26.74,29.0,X-LC,6.39,3.0,-0.30,1.68,0.80,XY24,NTT,AUTO
11,BATAINDIA,2096.00,-0.00,-38.08,118.36,35.20,94300.0,-48998.0,79672.0,4.40,34.0,X-SC,15.89,92.0,-0.52,0.59,1.20,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,0.41,-29.57,102.80,42.84,47068.0,-19220.0,45786.0,-53.72,42.0,X-SC,36.19,83.0,-0.41,0.34,3.59,XY24,NTT,MISC
15,CAMS,4762.00,-0.07,-80.18,529.73,24.79,264383.0,-201947.0,49909.0,-81.09,45.0,X-SC,0.94,86.0,-0.76,0.37,22.01,X40N,NTT,MISC
59,RELAXO,1176.00,-0.34,-47.51,190.16,52.31,144894.0,-68964.0,76196.0,-44.15,43.0,X-SC,6.66,91.0,-0.48,0.57,1.99,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.00,-38.08,118.36,35.20,94300.0,-48998.0,79672.0,4.40,34.0,X-SC,15.89,92.0,-0.52,0.59,1.20,X40,NTT,FOOTWEAR
22,DIXON,18931.72,2.18,-14.77,42.65,21.58,50941.0,-20700.0,119439.0,-57.05,35.0,X-MC,7.74,56.0,-0.41,0.89,7.45,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.68,-12.69,36.61,19.28,110587.0,-43887.0,302069.0,-23.95,63.0,X-LC,4.93,1.0,-0.40,2.24,11.67,X40,ATH,IT
41,INFY,2275.00,-0.29,8.46,42.79,54.87,145218.0,26485.0,339373.0,-15.65,61.0,X-LC,3.31,2.0,0.18,2.52,17.45,X40,BTT,IT
75,TMPV,600.00,-0.14,-17.97,73.34,42.18,166282.0,-49682.0,226728.0,-26.74,29.0,X-LC,6.39,3.0,-0.30,1.68,0.80,XY24,NTT,AUTO
81,VBL,671.64,-0.52,-3.85,41.10,35.67,124816.0,-12154.0,303688.0,-15.30,55.0,X-LC,4.44,4.0,-0.10,2.25,9.47,X40N,ATH,FMCG
43,ITC,452.00,-0.56,-2.11,12.82,10.44,25117.0,-4220.0,195918.0,-41.56,42.0,X-LC,1.84,5.0,-0.17,1.45,3.34,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,1.04,-37.67,113.21,32.89,54479.0,-29084.0,48122.0,-694.37,54.0,L-MC,5.86,259.0,-0.53,0.36,32.52,XR,NTT,BANKS
6,ASIANTILES,137.00,0.99,-15.06,113.90,81.67,90753.0,-14132.0,79678.0,7161.11,50.0,L-SC,13.61,271.0,-0.16,0.59,54.30,XR,NTT,CERAMICS
50,MASFIN,398.61,0.73,-1.32,23.68,22.05,22896.0,-1290.0,96690.0,-15.09,58.0,H-SC,6.96,164.0,-0.06,0.72,40.55,XR,ATH,FINANCE
13,BSOFT,831.70,0.26,-22.21,93.85,50.79,102679.0,-31245.0,109408.0,-0.27,67.0,H-SC,5.96,151.0,-0.30,0.81,27.39,XR,ATH,IT
39,INDIGOPNTS,1408.00,-0.04,-12.51,14.32,0.02,21869.0,-21841.0,152718.0,132.70,52.0,M-SC,15.82,234.0,-1.00,1.13,32.84,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.7,0.26,-22.21,93.85,50.79,102679.0,-31245.0,109408.0,-0.27,67.0,H-SC,5.96,151.0,-0.30,0.81,27.39,XR,ATH,IT
1,ABB,7934.0,-0.04,-3.85,51.40,45.57,129294.0,-10075.0,251544.0,-38.17,57.0,H-MC,4.80,120.0,-0.08,1.87,7.03,AR,NTT,ELECTRICAL
0,5PAISA,593.0,0.16,-41.36,92.53,12.90,118557.0,-90368.0,128128.0,88.70,43.0,H-SC,11.84,161.0,-0.76,0.95,6.54,OX40N,NTT,FINANCE
84,WIPRO,420.0,0.35,6.71,61.44,72.27,111090.0,11363.0,180811.0,-8.35,68.0,M-LC,2.02,99.0,0.10,1.34,13.30,XR,NTT,IT
56,RAJESHEXPO,518.0,-0.98,-63.19,171.84,0.07,86772.0,-86681.0,50496.0,1751.01,55.0,L-SC,2.40,268.0,-1.00,0.37,25.58,OX40N,NTT,JEWELLERY


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.79
1,25,45.69
2,50,76.63


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.04
MC    30.32
LC    24.60
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.34
X40      22.36
X40N     10.87
XR       10.13
XY25      9.39
AR        9.32
OX40N     7.75
SR        1.02
MH        0.78
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.61
X-MC    23.55
X-LC    21.06
M-SC    12.10
X-SC     6.91
H-MC     5.01
L-SC     1.42
M-MC     1.40
M-LC     1.34
H-LC     1.15
L-LC     1.05
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.29,-8.00,42.93
IT,12.81,-16.78,79.92
FINANCE,9.61,-19.74,70.34
ELECTRICAL,6.22,-11.99,53.01
MISC,6.06,-58.82,123.89
PAINTS,6.06,-13.04,29.83
INSURANCE,4.60,-2.06,38.03
PHARMA,4.10,-1.20,33.58
AUTO,3.43,-24.58,78.00


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3219643.0,21
AR,1315171.0,10
XR,1295482.0,13
X40,1026414.0,14
X40N,984721.0,9
OX40N,747163.0,10
XY25,376035.0,6
SR,283054.0,2
MH,78987.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3661704.0,25
M-SC,1429618.0,15
X-MC,1309806.0,16
X-SC,990051.0,8
X-LC,926067.0,11
H-MC,403408.0,3
L-SC,268511.0,3
M-LC,111090.0,1
H-LC,78013.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1277032.0      6
           AR         902065.0      5
M-SC       XY24       815024.0      6
H-SC       XR         774142.0      7
X-SC       X40N       507151.0      3
X-MC       X40        456871.0      7
           XY24       396826.0      3
X-LC       X40        381731.0      5
H-SC       OX40N      346424.0      4
M-SC       OX40N      313967.0      5
X-SC       XY24       295088.0      3
H-SC       SR         283054.0      2
X-MC       X40N       273878.0      4
X-LC       XY24       240763.0      2
H-MC       AR         208498.0      2
X-LC       X40N       203692.0      2
H-MC       XY24       194910.0      1
X-SC       X40        187812.0      2
X-MC       XY25       182231.0      2
L-SC       XR         181739.0      2
M-SC       XR         174032.0      2
           AR         126595.0      2
M-LC       XR         111090.0      1
X-LC       XY25        99881.0      2
L-SC       OX40N       86772.0      1
H-SC       MH          78987.0      1
H-LC       AR          78013.0      1
M-MC       XY25        56434.0      1
L-MC       XR          54479.0      1
L-LC       XY25        37489.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
